In [4]:
%load_ext autoreload
%autoreload 2

import spectral_unmixing_tools as el_spectral

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
pip install git+https://github.com/yourusername/yourrepositoryname.git


In [37]:
base_folder = "NIWOT_calibration_flight_08_2020"
el_spectral.resample_translation_to_other_sensors(base_folder)

Processing folder: NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance
Command executed successfully for sensor type: Landsat 5 TM
Standard Output: {'description': 'Resampled hyperspectral data', 'file type': 'ENVI Standard', 'sensor type': 'landsat', 'bands': 6, 'lines': '10782', 'samples': '1071', 'header offset': 0, 'data type': 4, 'interleave': 'bip', 'byte order': 0, 'wavelength units': 'Nanometers', 'wavelength': array([ 485.,  560.,  660.,  830., 1650., 2215.], dtype=float32)}

Command executed successfully for sensor type: Landsat 7 ETM+
Standard Output: {'description': 'Resampled hyperspectral data', 'file type': 'ENVI Standard', 'sensor type': 'landsat', 'bands': 6, 'lines': '10782', 'samples': '1071', 'header offset': 0, 'data type': 4, 'interleave': 'bip', 'byte order': 0, 'wavelength units': 'Nanometers', 'wavelength': array([ 477.,  560.,  662.,  835., 1648., 2206.], dtype=float32)}



KeyboardInterrupt: 

## Extract spectra as array

In [38]:
import rasterio
import numpy as np

class ENVIProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = None  # This will hold the raster data array
        self.file_type = "envi"
        # Other attributes...

    def load_data(self):
        """Loads the raster data from the file_path into self.data"""
        with rasterio.open(self.file_path) as src:
            self.data = src.read()  # Read all bands

    def get_chunk_from_extent(self, corrections=[], resample=False):
        self.load_data()  # Ensure data is loaded
        with rasterio.open(self.file_path) as src:
            bounds = src.bounds
            width, height = src.width, src.height
            col_start, line_start = 0, 0
            col_end, line_end = width, height

            # Assuming self.data is a 3D numpy array with dimensions [bands, rows, cols]
            chunk = self.data[:, line_start:line_end, col_start:col_end]
            
            # Apply any processing to chunk here...
            # For example, to demonstrate, flip chunk vertically
            chunk = np.flip(chunk, axis=1)
            
            return chunk



# Example usage:
# Replace 'your_envi_file_path' with the actual path to your ENVI file
raster_path = "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectance_resample_Landsat_5_TM.img"  # Update this to your actual raster file path
processor = ENVIProcessor(raster_path)
chunk = processor.get_chunk_from_extent(corrections=['some_correction'], resample=False)



In [39]:
chunk.shape

(6, 10782, 1071)

In [21]:
pd.DataFrame(chunk)

ValueError: Must pass 2-d input. shape=(6, 10782, 1071)

In [4]:
## flatten array into a 2D df


In [40]:
import pandas as pd
import numpy as np

def flatten_array_to_dataframe(array):
    """
    Flattens a 3D numpy array into a pandas DataFrame.
    
    Parameters:
    - array: A 3D numpy array of shape (bands, rows, cols).
    
    Returns:
    - A pandas DataFrame where each row represents a pixel across all bands.
    """
    # Ensure the input is a 3D numpy array
    if len(array.shape) != 3:
        raise ValueError("Input array must be 3-dimensional.")
    
    bands, rows, cols = array.shape
    # Reshape the array to have pixels as rows and bands as columns
    reshaped_array = array.reshape(bands, -1).T  # Transpose to make bands as columns
    
    # Create a DataFrame from the reshaped array
    df = pd.DataFrame(reshaped_array, columns=[f'Band_{i+1}' for i in range(bands)])
    
    # Optionally, add pixel row and column indices
    pixel_indices = np.indices((rows, cols)).reshape(2, -1).T  # 2D array of row,col indices for each pixel
    df['Pixel_Row'] = pixel_indices[:, 0]
    df['Pixel_Col'] = pixel_indices[:, 1]
    
    return df

# Example usage:
# Assuming 'chunk' is your 3D numpy array of shape (426, 11138, 1031)

df = flatten_array_to_dataframe(chunk)



In [41]:
df

,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Pixel_Row,Pixel_Col
0,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,0,0
1,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,0,1
2,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,0,2
3,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,0,3
4,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,0,4
...,...,...,...,...,...,...,...,...
11547517,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,10781,1066
11547518,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,10781,1067
11547519,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,10781,1068
11547520,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,10781,1069


## Reshape data frame and change labels

In [ ]:
import numpy as np

def iterate_flatten_melt_array(array):
    """
    Generator to iterate over a 3D numpy array and yield "melted" data.
    
    Parameters:
    - array: A 3D numpy array of shape (bands, rows, cols).
    
    Yields:
    - Tuple of (Pixel_Row, Pixel_Col, Band_ID, Wavelength) for each pixel-band combination.
    """
    bands, rows, cols = array.shape
    
    for band in range(bands):
        for row in range(rows):
            for col in range(cols):
                yield (row, col, f'Band_{band+1}', array[band, row, col])

# Example usage
chunk = np.random.rand(426, 11138, 1031)  # Replace with your actual data

# To demonstrate or test the generator, you can iterate through a small portion of it
for i, data_point in enumerate(iterate_flatten_melt_array(chunk)):
    print(data_point)
    if i > 100:  # Adjust this condition to control how many items you want to print
        break


In [ ]:
import csv

# Open a CSV file for writing
with open('melted_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Pixel_Row', 'Pixel_Col', 'Band_ID', 'Wavelength'])  # Write header

    # Write each data point
    for data_point in iterate_flatten_melt_array(chunk):
        writer.writerow(data_point)
import numpy as np
import pandas as pd

def batch_flatten_melt_array(array, batch_size=1000000):
    """
    Generator to iterate over a 3D numpy array and yield batches of "melted" data.
    
    Parameters:
    - array: A 3D numpy array of shape (bands, rows, cols).
    - batch_size: The number of rows in each batch.
    
    Yields:
    - A DataFrame containing a batch of melted data.
    """
    bands, rows, cols = array.shape
    total_pixels = rows * cols
    num_batches = (total_pixels + batch_size - 1) // batch_size  # Ceiling division to get the number of batches
    
    for batch in range(num_batches):
        batch_data = []
        start_index = batch * batch_size
        end_index = min(start_index + batch_size, total_pixels)
        
        for index in range(start_index, end_index):
            row = index // cols
            col = index % cols
            for band in range(bands):
                batch_data.append((row, col, f'Band_{band+1}', array[band, row, col]))
                
        batch_df = pd.DataFrame(batch_data, columns=['Pixel_Row', 'Pixel_Col', 'Band_ID', 'Wavelength'])
        yield batch_df

# Example usage
chunk = np.random.rand(426, 11138, 1031)  # Replace with your actual data

# Iterate through each batch and process
for i, batch_df in enumerate(batch_flatten_melt_array(chunk)):
    print(f"Processing batch {i+1}")
    # Process the batch_df here
    # For example, you could save each batch to a separate CSV file
    batch_df.to_csv(f'melted_data_batch_{i+1}.csv', index=False)
    if i == 0:  # For demonstration, break after processing the first batch
        break
